In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style1/128/b1/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 4000
start_epoch = 0
train_batch = 12
test_batch = 200
lr = 0.01
schedule = [200, 500, 1000, 2000, 3000]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style1/128/b1/to_pggan/cutmix3' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)
random_crop = (144, 144)

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

# iterative training
feedback = 0
iter_time = [1000, 2000, 3000]

# cutmix
cm_prob = 0.5
cm_prob_init = 0.01
cm_prob_low = 0.99

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(target_dir, '100_shot')
source_train_dir = os.path.join(target_dir, '100_shot_style1_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(random_crop),
    transforms.RandomCrop(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
#     transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(random_crop),
    transforms.CenterCrop(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
source_train_loader = DataLoader(datasets.ImageFolder(source_train_dir, transform=val_aug),
                                batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes)
student_model = EfficientNet.from_name(model_name, num_classes=num_classes)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style1/128/b1/checkpoint.pth.tar'


In [9]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in student_model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_bn2): GroupNorm(4, 16, ep

In [11]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [12]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [13]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Loss

In [14]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
# optimizer = optim.Adam(model.parameters(), weight_decay=0)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=50, after_scheduler=scheduler_cosine)

In [15]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.', 'Source Loss', 'Source ACC.'])

# Train

In [16]:
def train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    student_model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    source_set = []
    for inputs, targets in source_train_loader:
        source_set.append((inputs, targets))
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
        
        # cutmix
        source_inputs, source_targets = source_set[batch_idx]
        source_inputs, source_targets = source_inputs.cuda(), source_targets.cuda()

        prob_delta = cm_prob_init - cm_prob_low
        prob_step = epoch / (epochs+1) * prob_delta
        lam = cm_prob_init - prob_step

        rand_index = torch.randperm(inputs.size()[0]).cuda()
        st = source_targets[rand_index]
        tt = targets[rand_index]
        equal_index = rand_index[st == tt]
        diff_index = rand_index[st != tt]
        
        bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
        inputs[equal_index, :, bbx1:bbx2, bby1:bby2] = source_inputs[equal_index, :, bbx1:bbx2, bby1:bby2]
        inputs[diff_index, :, bbx1:bbx2, bby1:bby2] = inputs[diff_index, :, bbx1:bbx2, bby1:bby2]
        
        lam_ratio = 1 - ((bbx2 - bbx1)*(bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))
        targets_a = targets
        targets_b = targets
        targets_b[diff_index] = source_targets[diff_index]
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets_a)*lam_ratio + criterion(outputs, targets_b)*(1 - lam_ratio)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
            
            
        # compute output
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
#         if batch_idx % 10 == 0:
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
             batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [17]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(model)
        loss_sp = reg_l2sp(model)
        loss = loss_main + sp_alpha*loss_sp + sp_beta*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [18]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [19]:
for epoch in range(start_epoch, epochs):
    # teacher feedback
    if epoch in iter_time:
        print("iterative training: feedback {}".format(epoch))
        teacher_model.load_state_dict(student_model.state_dict())
        teacher_model_weights = {}
        for name, param in teacher_model.named_parameters():
            teacher_model_weights[name] = param.detach()
    
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc = train(train_loader, source_train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    
    if epoch % 50 == 0:
        test_loss, test_acc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
        source_loss, source_acc = test(val_source_loader, student_model, criterion, epoch, use_cuda)

        logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc, source_loss, source_acc])
        is_best = test_acc > best_acc
        best_acc = max(test_acc, best_acc)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict' : student_model.state_dict(),
            'acc': test_acc,
            'best_acc': best_acc,
            'optimizer': optimizer.state_dict(),
        }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()


Epoch: [1 | 4000] LR: 0.010000
17/17 Data:0.000 | Batch:0.286 | Total:0:00:10 | ETA:0:00:01 | Loss:5.076074041426182 | top1:54.16666793823242


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.001 | Batch:0.771 | Total:0:00:43 | ETA:0:00:00 | Loss:9.549737505452283 | top1:50.000003814697266
39/39 Data:0.000 | Batch:0.314 | Total:0:00:20 | ETA:0:00:00 | Loss:9.452399449470716 | top1:50.11538314819336

Epoch: [2 | 4000] LR: 0.010600
17/17 Data:0.000 | Batch:0.753 | Total:0:00:12 | ETA:0:00:01 | Loss:10.91431075334549 | top1:45.833335876464844

Epoch: [3 | 4000] LR: 0.011200
17/17 Data:0.000 | Batch:0.599 | Total:0:00:10 | ETA:0:00:01 | Loss:10.087122559547424 | top1:53.125

Epoch: [4 | 4000] LR: 0.011800
17/17 Data:0.000 | Batch:0.602 | Total:0:00:11 | ETA:0:00:01 | Loss:7.45037117600441 | top1:53.645835876464844

Epoch: [5 | 4000] LR: 0.012400
17/17 Data:0.001 | Batch:0.620 | Total:0:00:09 | ETA:0:00:01 | Loss:5.632320165634155 | top1:66.14583587646484

Epoch: [6 | 4000] LR: 0.013000
17/17 Data:0.000 | Batch:0.282 | Total:0:00:09 | ETA:0:00:01 | Loss:4.097117871046066 | top1:67.70833587646484

Epoch: [7 | 4000] LR: 0.013600
17/17 Data:0.000 | Batch:0.541 | Tota

17/17 Data:0.000 | Batch:0.383 | Total:0:00:07 | ETA:0:00:01 | Loss:59.120250046253204 | top1:57.8125

Epoch: [59 | 4000] LR: 0.040000
17/17 Data:0.001 | Batch:0.531 | Total:0:00:09 | ETA:0:00:01 | Loss:303.08739471435547 | top1:63.54166793823242

Epoch: [60 | 4000] LR: 0.040000
17/17 Data:0.002 | Batch:0.489 | Total:0:00:08 | ETA:0:00:01 | Loss:164.60838890075684 | top1:47.395835876464844

Epoch: [61 | 4000] LR: 0.040000
17/17 Data:0.000 | Batch:0.323 | Total:0:00:07 | ETA:0:00:01 | Loss:36.053096652030945 | top1:50.520835876464844

Epoch: [62 | 4000] LR: 0.039999
17/17 Data:0.000 | Batch:0.334 | Total:0:00:07 | ETA:0:00:01 | Loss:4.224781237542629 | top1:56.770835876464844

Epoch: [63 | 4000] LR: 0.039999
17/17 Data:0.002 | Batch:0.294 | Total:0:00:06 | ETA:0:00:01 | Loss:1.6952819637954235 | top1:54.16666793823242

Epoch: [64 | 4000] LR: 0.039999
17/17 Data:0.000 | Batch:0.349 | Total:0:00:07 | ETA:0:00:01 | Loss:2.868263766169548 | top1:55.208335876464844

Epoch: [65 | 4000] LR: 0.

17/17 Data:0.000 | Batch:0.382 | Total:0:00:07 | ETA:0:00:01 | Loss:96.33856917917728 | top1:52.083335876464844

Epoch: [115 | 4000] LR: 0.039976
17/17 Data:0.000 | Batch:0.466 | Total:0:00:08 | ETA:0:00:01 | Loss:328.2356815338135 | top1:62.5

Epoch: [116 | 4000] LR: 0.039975
17/17 Data:0.000 | Batch:0.386 | Total:0:00:07 | ETA:0:00:01 | Loss:171.55954122543335 | top1:64.0625

Epoch: [117 | 4000] LR: 0.039974
17/17 Data:0.000 | Batch:0.303 | Total:0:00:06 | ETA:0:00:01 | Loss:53.01539599895477 | top1:67.70833587646484

Epoch: [118 | 4000] LR: 0.039973
17/17 Data:0.000 | Batch:0.410 | Total:0:00:06 | ETA:0:00:01 | Loss:23.744861841201782 | top1:58.85416793823242

Epoch: [119 | 4000] LR: 0.039972
17/17 Data:0.000 | Batch:0.307 | Total:0:00:06 | ETA:0:00:01 | Loss:73.05990862846375 | top1:60.9375

Epoch: [120 | 4000] LR: 0.039971
17/17 Data:0.000 | Batch:0.429 | Total:0:00:07 | ETA:0:00:01 | Loss:460.38632678985596 | top1:61.97916793823242

Epoch: [121 | 4000] LR: 0.039971
17/17 Data:0.0

...

17/17 Data:0.002 | Batch:0.312 | Total:0:00:05 | ETA:0:00:01 | Loss:0.7873021364212036 | top1:47.91666793823242

Epoch: [3755 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.248 | Total:0:00:04 | ETA:0:00:01 | Loss:0.738671638071537 | top1:55.72916793823242

Epoch: [3756 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.245 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8555274941027164 | top1:42.1875

Epoch: [3757 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.296 | Total:0:00:04 | ETA:0:00:01 | Loss:0.783212635666132 | top1:53.645835876464844

Epoch: [3758 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.240 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7527087740600109 | top1:57.8125

Epoch: [3759 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.272 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7290556412190199 | top1:58.85416793823242

Epoch: [3760 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.234 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7125173471868038 | top1:58.85416793823242

Epoch: [3761 | 4000] LR: 0

17/17 Data:0.002 | Batch:0.280 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7510880175977945 | top1:56.770835876464844

Epoch: [3811 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.250 | Total:0:00:04 | ETA:0:00:01 | Loss:0.789892440661788 | top1:51.5625

Epoch: [3812 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.270 | Total:0:00:05 | ETA:0:00:01 | Loss:0.8188463002443314 | top1:50.0

Epoch: [3813 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.296 | Total:0:00:05 | ETA:0:00:01 | Loss:0.7645922899246216 | top1:50.520835876464844

Epoch: [3814 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.301 | Total:0:00:04 | ETA:0:00:01 | Loss:0.769348967820406 | top1:54.16666793823242

Epoch: [3815 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.272 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7216598428785801 | top1:61.458335876464844

Epoch: [3816 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.278 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7920954935252666 | top1:52.083335876464844

Epoch: [3817 | 4000] LR: 0

17/17 Data:0.001 | Batch:0.283 | Total:0:00:05 | ETA:0:00:01 | Loss:0.7949326112866402 | top1:47.395835876464844

Epoch: [3867 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.289 | Total:0:00:04 | ETA:0:00:01 | Loss:0.752121027559042 | top1:53.645835876464844

Epoch: [3868 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.282 | Total:0:00:04 | ETA:0:00:01 | Loss:0.816580031067133 | top1:46.875

Epoch: [3869 | 4000] LR: 0.000000
17/17 Data:0.003 | Batch:0.304 | Total:0:00:05 | ETA:0:00:01 | Loss:0.8303864300251007 | top1:48.958335876464844

Epoch: [3870 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.290 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7671749107539654 | top1:54.6875

Epoch: [3871 | 4000] LR: 0.000000
17/17 Data:0.004 | Batch:0.320 | Total:0:00:05 | ETA:0:00:01 | Loss:0.7435929775238037 | top1:53.645835876464844

Epoch: [3872 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.255 | Total:0:00:06 | ETA:0:00:01 | Loss:0.8203037679195404 | top1:48.958335876464844

Epoch: [3873 | 4000] LR

17/17 Data:0.001 | Batch:0.250 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8001391924917698 | top1:47.395835876464844

Epoch: [3923 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.262 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7963469438254833 | top1:49.47916793823242

Epoch: [3924 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.290 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8119918294250965 | top1:45.3125

Epoch: [3925 | 4000] LR: 0.000000
17/17 Data:0.003 | Batch:0.287 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7562246732413769 | top1:56.25

Epoch: [3926 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.287 | Total:0:00:05 | ETA:0:00:01 | Loss:0.7609799318015575 | top1:53.645835876464844

Epoch: [3927 | 4000] LR: 0.000000
17/17 Data:0.003 | Batch:0.279 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7583971861749887 | top1:53.125

Epoch: [3928 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.292 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7342981174588203 | top1:59.375

Epoch: [3929 | 4000] LR: 0.000000
17/17 Data:0.

17/17 Data:0.002 | Batch:0.281 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7626681514084339 | top1:53.645835876464844

Epoch: [3979 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.273 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8274255692958832 | top1:52.60416793823242

Epoch: [3980 | 4000] LR: 0.000000
17/17 Data:0.001 | Batch:0.283 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7700260542333126 | top1:52.60416793823242

Epoch: [3981 | 4000] LR: 0.000000
17/17 Data:0.003 | Batch:0.291 | Total:0:00:04 | ETA:0:00:01 | Loss:0.811802189797163 | top1:50.520835876464844

Epoch: [3982 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.268 | Total:0:00:04 | ETA:0:00:01 | Loss:0.7237286865711212 | top1:57.29166793823242

Epoch: [3983 | 4000] LR: 0.000000
17/17 Data:0.000 | Batch:0.268 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8045503869652748 | top1:54.16666793823242

Epoch: [3984 | 4000] LR: 0.000000
17/17 Data:0.002 | Batch:0.278 | Total:0:00:04 | ETA:0:00:01 | Loss:0.8305970765650272 | top1:48.4375

Epoch: [3985 |